In [1]:
%load_ext autoreload
%autoreload 2
import sys

# insert at 1, 0 is the script path (or ‘’ in REPL)
sys.path.insert(1, '../')
import numpy as np
import torch
from torch.utils.data import DataLoader
from bayesian_neural_net import CLSTM_cell
from bayesian_neural_net import ConvCell
from bayesian_neural_net import Encoder
from bayesian_neural_net import Decoder
from bayesian_neural_net import ED
from visualization import plot_spatio_temporal_data
from simulation_dataset import DatasetDstm
from simulation_dataset import DatasetDstmDecoderWithoutInput
from simulation_dataset import DatasetDstmEncoderWithoutInput
from simulation_dataset import DatasetDstm3
from simulation_dataset_datamodule import DatasetDataModule
from trainer import LightningED
from visualization import plot_spatio_temporal_data
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

# DSTM

In [ ]:
n = 5
T = 15
gamma = 0.3
l = 1
offset = 0
total = 200
mask = np.ones([total, T]) * np.array([1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1]) == 1

baseline_underlying = 10
baseline_precipitation = 4
data = DatasetDstm(n, T, gamma, l, offset, total, mask, baseline_underlying, baseline_precipitation)
data_module = DatasetDataModule(data, 1, 0.5)

rnns = [CLSTM_cell(shape=(5, 5), input_channels=2, filter_size=1, num_features=16, dropout_rate=0.5), CLSTM_cell(shape=(5, 5), input_channels=16, filter_size=1, num_features=16, dropout_rate=0.5)]
encoder_net = Encoder(rnns)
rnns = [CLSTM_cell(shape=(5, 5), input_channels=1, filter_size=1, num_features=16, dropout_rate=0.5), CLSTM_cell(shape=(5, 5), input_channels=16, filter_size=1, num_features=16, dropout_rate=0.5)]
cnn = ConvCell(in_channels=32, out_channels=1, kernel_size=1, stride=1, padding=0)
decoder_net = Decoder(rnns, cnn)
ED_net = ED(encoder_net, decoder_net)

mc_dropout = 5
learning_rate = 1e-3
max_epoch = 20
model = LightningED(ED_net, mc_dropout, learning_rate)
logger = TensorBoardLogger('tb_logs', name='Bayesian_ConvLSTM')
# trainer = pl.Trainer(auto_lr_find=True, logger=logger)
# trainer = pl.Trainer(logger=logger, fast_dev_run=True)
if torch.cuda.is_available():
    trainer = pl.Trainer(logger=logger, max_epochs=max_epoch, gpus=1)
else:
     trainer = pl.Trainer(logger=logger, max_epochs=max_epoch) 
    




# trainer.tune(model, dm)
trainer.fit(model, data_module)

# DSTM decoder without inputs

In [ ]:

n = 5
T = 15
gamma = 0.3
l = 1
offset = 0
total = 200
mask = np.ones([total, T]) * np.array([1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1]) == 1

baseline_underlying = 10
baseline_precipitation = 4
data = DatasetDstmDecoderWithoutInput(n, T, gamma, l, offset, total, mask, baseline_underlying, baseline_precipitation)
data_module = DatasetDataModule(data, 1, 0.5)

rnns = [CLSTM_cell(shape=(5, 5), input_channels=2, filter_size=1, num_features=16, dropout_rate=0.5), CLSTM_cell(shape=(5, 5), input_channels=16, filter_size=1, num_features=16, dropout_rate=0.5)]
encoder_net = Encoder(rnns)
rnns = [CLSTM_cell(shape=(5, 5), input_channels=1, filter_size=1, num_features=16, dropout_rate=0.5), CLSTM_cell(shape=(5, 5), input_channels=16, filter_size=1, num_features=16, dropout_rate=0.5)]
cnn = ConvCell(in_channels=32, out_channels=1, kernel_size=1, stride=1, padding=0)
decoder_net = Decoder(rnns, cnn)
ED_net = ED(encoder_net, decoder_net)

mc_dropout = 5
learning_rate = 1e-3
max_epoch = 20
model = LightningED(ED_net, mc_dropout, learning_rate)
logger = TensorBoardLogger('tb_logs', name='Bayesian_ConvLSTM')
# trainer = pl.Trainer(auto_lr_find=True, logger=logger)
# trainer = pl.Trainer(logger=logger, fast_dev_run=True)
# trainer = pl.Trainer(logger=logger, max_epochs=args.max_epoch, gpus=1)
if torch.cuda.is_available():
    trainer = pl.Trainer(logger=logger, max_epochs=max_epoch, gpus=1)
else:
     trainer = pl.Trainer(logger=logger, max_epochs=max_epoch) 



# trainer.tune(model, dm)
trainer.fit(model, data_module)

# Dstm encoder without inputs

In [ ]:

n = 5
T = 15
gamma = 0.3
l = 1
offset = 0
total = 200
mask = np.ones([total, T]) * np.array([1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1]) == 1

baseline_underlying = 10
baseline_precipitation = 4
data = DatasetDstmEncoderWithoutInput(n, T, gamma, l, offset, total, mask, baseline_underlying, baseline_precipitation)
data_module = DatasetDataModule(data, 1, 0.5)

rnns = [CLSTM_cell(shape=(5, 5), input_channels=2, filter_size=1, num_features=16, dropout_rate=0.5), CLSTM_cell(shape=(5, 5), input_channels=16, filter_size=1, num_features=16, dropout_rate=0.5)]
encoder_net = Encoder(rnns)
rnns = [CLSTM_cell(shape=(5, 5), input_channels=1, filter_size=1, num_features=16, dropout_rate=0.5), CLSTM_cell(shape=(5, 5), input_channels=16, filter_size=1, num_features=16, dropout_rate=0.5)]
cnn = ConvCell(in_channels=32, out_channels=1, kernel_size=1, stride=1, padding=0)
decoder_net = Decoder(rnns, cnn)
ED_net = ED(encoder_net, decoder_net)

mc_dropout = 5
learning_rate = 1e-3
max_epoch = 20
model = LightningED(ED_net, mc_dropout, learning_rate)
logger = TensorBoardLogger('tb_logs', name='Bayesian_ConvLSTM')
# trainer = pl.Trainer(auto_lr_find=True, logger=logger)
# trainer = pl.Trainer(logger=logger, fast_dev_run=True)
# trainer = pl.Trainer(logger=logger, max_epochs=args.max_epoch, gpus=1)
if torch.cuda.is_available():
    trainer = pl.Trainer(logger=logger, max_epochs=max_epoch, gpus=1)
else:
     trainer = pl.Trainer(logger=logger, max_epochs=max_epoch) 



# trainer.tune(model, dm)
trainer.fit(model, data_module)